In [1]:
from tools import *
from load_data import *
from scraper import *

In [2]:
import os
import re

In [3]:
import pandas as pd

In [4]:
from tqdm import tqdm_notebook as tqdm

# Core Data

In [5]:
data = pd.read_csv('../model/5_2_2020_data.csv')

In [6]:
subset = subset_df(data, 'file_name')

In [7]:
by_race = [subset_df(i, 'sheet_name') for i in subset]
by_race = unlist(by_race)

In [8]:
by_race_dict = dict()
for i in by_race:
    date = i.file_name.unique()[0]
    race = i.sheet_name.unique()[0]
    id_ = '_'.join(format_date_race(date, race))
    by_race_dict[id_] = i[[j for j in list(i) if ('sum' not in j) and ('total' not in j) and (j not in ['file_name','sheet_name','result'])]].reset_index(drop=True)

# HKJC Data

In [9]:
path = '../../extra_size_files/data/5_1_2020_info'

In [10]:
all_dir = os.listdir(path)

In [11]:
rp = [i for i in all_dir if 'rp' in i]
rt = [i for i in all_dir if 'rt' in i]
result = [i for i in all_dir if 'result' in i]

## Race tab

In [12]:
# get race tab
race_tab = list()
for i in rt:
    with open(os.path.join(path, i)) as f:
        wow = [i.strip() for i in f.readlines()]
    race_tab.append(wow)

In [13]:
id_rt = [re.sub(r'_rt.txt', '', i) for i in rt]

### Length class

In [14]:
def length_class_detector(str_):
    """ detect length and class
    """
    detect = re.findall(r'[0-9]{3,4}M', str_)
    if detect:
        return True
    else:
        return False

In [15]:
length_class = [[i for i in j if length_class_detector(i)] for j in race_tab]

In [16]:
# create df for length and class
length_class_col = ['Class','Length','Rating','Class_Note']
length_class_df = pd.DataFrame([unlist([i.split(' - ') for i in j]) for j in length_class], columns=length_class_col)

In [17]:
length_class_df.head()

,Class,Length,Rating,Class_Note
0,Class 4,1400M,(60-40),None
1,Class 2,1400M,(100-80),None
2,Class 4,1800M,(60-35),None
3,Class 2,1200M,(105-80),None
4,Class 3,1000M,(80-60),None


### Going & Course

In [18]:
def extract_going_course(in_list):
    """ extract going and course, use it to loop through race tab
    """
    info = [i for i,j in enumerate(in_list) if ' :' in j] + [len(in_list)]
    info_extracted = [in_list[info[i]:info[i+1]] for i in range(len(info)-1)]
    info_dict = {re.findall(r'\w{1,10}',i[0])[0]:i[1:] for i in info_extracted}
    return info_dict['Going'], [i for i in info_dict['Course'] if 'HK$' not in i]

In [19]:
all_going_course = [extract_going_course(i) for i in race_tab]

In [20]:
going_df = pd.DataFrame([i[0] for i in all_going_course], columns=['Going','Handicap'])

In [21]:
course_df = pd.DataFrame([i[1] for i in all_going_course], columns=['Course'])

### Combine Length, Class, Going, and Course

In [22]:
race_tab_df = pd.concat([length_class_df, going_df, course_df], 1)
race_tab_df['id'] = id_rt

## Race Performance & Core data

In [23]:
race_per_dict = {re.sub(r'_rp.csv', '', i):pd.read_csv(os.path.join(path, i)) for i in rp}

In [24]:
# clean Horse No. issue, ' 11 ', etc
race_per_dict_cleaned = dict()
for i in race_per_dict:
    df = race_per_dict[i]
    df['horse_num'] = [str(i).strip() for i in df['Horse No.'].values]
    race_per_dict_cleaned[i] = df[[i for i in list(df) if i != 'Horse No.']]

In [25]:
core_performance = dict()
for i in tqdm(id_rt):
    a = race_per_dict_cleaned[i].astype({'horse_num':'str'})
    b = by_race_dict[i].astype({'horse_num':'str'})

    merged = a.merge(b, left_on='horse_num', right_on='horse_num', how='outer')
    cleaned = merged[[i for i in list(merged) if i != 'horse_num']]
    cleaned['race_id'] = [i]*cleaned.shape[0]

    core_performance[i] = cleaned

# All data

In [26]:
vote_info_result = pd.concat(core_performance.values())

In [27]:
all_data = vote_info_result.merge(race_tab_df, left_on='race_id', right_on='id')
all_data = all_data[[i for i in list(all_data) if i != 'id']]

## Clean all data

In [28]:
all_data.head(1)

,Plc.,Horse,Jockey,Trainer,Actual Wt.,Declar. Horse Wt.,Draw,LBW,RunningPosition,Finish Time,...,W.H. FixOdd 60,S.B.Fix Odd 60,race_id,Class,Length,Rating,Class_Note,Going,Handicap,Course
0,1,ULTIMATE GLORY(S235),Z Purton,K L Man,132.0,1278,7,-,13 13 12 1,1:21.93,...,NaN,NaN,2019_05_11_5,Class 4,1400M,(60-40),None,GOOD,FRIENDSHIP BRIDGE HANDICAP,"TURF - ""C"" Course"


## Plc.

In [29]:
# for i in list(all_data):
#     print(i)
#     print(all_data[i].unique())

In [30]:
col_ = list(all_data)[0]

In [31]:
all_data[col_].unique()

array(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', 'WV', 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14,
       '3 DH', 'WX', nan, '5 DH', '2 DH', 'WX-A', 'WV-A', '8 DH', '9 DH',
       'UR', '7 DH', 'FE', 'TNP', 'DNF', 'PU', '10 DH', '12 DH', '1 DH',
       '11 DH', '4 DH', 'WXNR', '6 DH'], dtype=object)

In [32]:
def fix_plc(input_):
    if isinstance(input_, str):
        return input_
    elif isinstance(input_, (int,float)) and not np.isnan(input_):
        return str(int(input_))
    else:
        return input_

In [33]:
all_data[col_] = [fix_plc(i) for i in all_data[col_]]

## Replace abnormal nan

In [34]:
replace = [' --- ', '---','-']

In [35]:
all_data = all_data.replace(replace, np.nan)

## Fix oriental dtypes

In [36]:
all_data = all_data.astype({i:j for i,j in zip(['oriental_1','oriental_2','oriental_3'], ['float']*3)})

## Fix Length from "2000M" to 2000

In [37]:
all_data['Length'] = [float(re.findall(r'^[0-9]{3,4}', i)[0]) for i in all_data.Length]

## FIx 'Declar. Horse Wt.', 'Draw' dytpes

In [38]:
all_data = all_data.astype({'Declar. Horse Wt.':'float', 'Draw':'float', 'Win Odds':'float'})

## Fix 'Draw' dtypes

In [39]:
all_data.dtypes

Plc.                  object
Horse                 object
Jockey                object
Trainer               object
Actual Wt.           float64
Declar. Horse Wt.    float64
Draw                 float64
LBW                   object
RunningPosition       object
Finish Time           object
Win Odds             float64
oriental_1           float64
oriental_2           float64
oriental_3           float64
apple_1              float64
apple_2              float64
apple_3              float64
pro_1                float64
pro_2                float64
pro_3                float64
w_h_odd              float64
s_b_odd              float64
W.H. FixOdd 60       float64
S.B.Fix Odd 60       float64
race_id               object
Class                 object
Length               float64
Rating                object
Class_Note            object
Going                 object
Handicap              object
Course                object
dtype: object

# Export to csv

In [40]:
all_data.to_csv('../model/model_data_5_2.csv', index=False)